In [1]:
import pandas as pd
import pyarrow

# Cargar dataset principal
app = pd.read_parquet("../datos_examen/application_.parquet")

app.shape

(307511, 122)

# Paso 2.1 - Preparación de datos

Eliminación de columnas con alto porcentaje de valores nulos

**Celda 1 - Cargar librerías y datos**

In [2]:
import pandas as pd
import numpy as np

# Cargar dataset principal
app = pd.read_parquet("../datos_examen/application_.parquet")

app.shape

(307511, 122)

**Celda 2 - Calcular porcentaje de valores nulos**

In [3]:
# Porcentaje de valores nulos por columna
missing_pct = app.isna().mean() * 100

missing_pct.sort_values(ascending=False).head(20)

COMMONAREA_AVG              69.872297
COMMONAREA_MODE             69.872297
COMMONAREA_MEDI             69.872297
NONLIVINGAPARTMENTS_MEDI    69.432963
NONLIVINGAPARTMENTS_MODE    69.432963
NONLIVINGAPARTMENTS_AVG     69.432963
FONDKAPREMONT_MODE          68.386172
LIVINGAPARTMENTS_AVG        68.354953
LIVINGAPARTMENTS_MEDI       68.354953
LIVINGAPARTMENTS_MODE       68.354953
FLOORSMIN_MODE              67.848630
FLOORSMIN_AVG               67.848630
FLOORSMIN_MEDI              67.848630
YEARS_BUILD_AVG             66.497784
YEARS_BUILD_MODE            66.497784
YEARS_BUILD_MEDI            66.497784
OWN_CAR_AGE                 65.990810
LANDAREA_MEDI               59.376738
LANDAREA_AVG                59.376738
LANDAREA_MODE               59.376738
dtype: float64

**Celda 3 - Definir criterio de eliminación**

In [4]:
# Columnas con más de 60% de valores nulos
cols_to_drop = missing_pct[missing_pct > 60].index

len(cols_to_drop)

17

**Celda 4 - Eliminar columnas**

In [5]:
# Eliminar columnas con alto porcentaje de nulos
app_clean = app.drop(columns=cols_to_drop)

app_clean.shape

(307511, 105)

**Celda 5 - Comentario**

### Eliminación de columnas con valores nulos

Se eliminaron las variables que presentan más del **60% de valores faltantes**, ya que su alta ausencia de información podría afectar negativamente el entrenamiento del modelo y no aportan suficiente valor predictivo.

Esta decisión reduce la dimensionalidad del dataset y mejora la calidad de los datos para las siguientes etapas.

# Paso 2.2 - Imputación de valores faltantes

Imputar valores faltantes SIN eliminar filas y sin usar información del TARGET.

**Celda 1 - Separar columnas numéricas y categóricas**

In [7]:
# Separar columnas numéricas y categóricas
num_cols = app_clean.select_dtypes(include=["int64", "float64"]).columns
cat_cols = app_clean.select_dtypes(include=["object"]).columns

len(num_cols), len(cat_cols)

(90, 15)

**Celda 2 - Imputar variables numéricas (mediana)**

In [9]:
# Imputación de valores nulos en variables numéricas (mediana)
for col in num_cols:
    app_clean[col] = app_clean[col].fillna(app_clean[col].median())

# Verificar nulos restantes en numéricas
app_clean[num_cols].isna().sum().sum()

np.int64(0)

**Celda 3 - Imputar variables categóricas (moda)**

In [11]:
# Imputación de valores nulos en variables categóricas (moda)
for col in cat_cols:
    app_clean[col] = app_clean[col].fillna(app_clean[col].mode()[0])

# Verificar nulos restantes en categóricas
app_clean[cat_cols].isna().sum().sum()

np.int64(0)

**Celda 4 - Verificación final**

In [12]:
# Verificar que no existan valores nulos
app_clean.isna().sum().sum()

np.int64(0)

**Celda 5 - Comentario**

### Imputación de valores faltantes

Las variables numéricas fueron imputadas utilizando la **mediana**, debido a su robustez frente a valores extremos.  
Las variables categóricas fueron imputadas utilizando la **moda**, al representar la categoría más frecuente.

Este proceso permite conservar el total de registros y asegurar que el dataset esté completamente definido para las etapas de modelado.

# Paso 2.3 - Preparar dataset final para modelado

**Celda 1 - Separar X e y**

In [13]:
# Paso 2.3 - Separación de variables (X, y)

TARGET_COL = "TARGET"

X = app_clean.drop(columns=[TARGET_COL])
y = app_clean[TARGET_COL]

X.shape, y.shape

((307511, 104), (307511,))

**Celda 2 - One-Hot Encoding (categóricas)**

In [14]:
# One-Hot Encoding para variables categóricas
X_encoded = pd.get_dummies(X, drop_first=True)

X_encoded.shape

(307511, 210)

**Celda 3 - Asegurar que todo sea numérico y sin NaNs**

In [15]:
# Verificación final: sin nulos y todo numérico
X_encoded.isna().sum().sum(), X_encoded.dtypes.head()

(np.int64(0),
 SK_ID_CURR            int64
 CNT_CHILDREN          int64
 AMT_INCOME_TOTAL    float64
 AMT_CREDIT          float64
 AMT_ANNUITY         float64
 dtype: object)

**Celda 4 - Train/Test Split estratificado (por desbalance del TARGET)**

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape, y_train.value_counts(normalize=True)*100

((246008, 210),
 (61503, 210),
 TARGET
 0    91.927092
 1     8.072908
 Name: proportion, dtype: float64)

**Celda 5 - Guardar datasets**

In [17]:
import os

os.makedirs("../artifacts", exist_ok=True)

X_train.to_parquet("../artifacts/X_train.parquet")
X_test.to_parquet("../artifacts/X_test.parquet")
y_train.to_frame("TARGET").to_parquet("../artifacts/y_train.parquet")
y_test.to_frame("TARGET").to_parquet("../artifacts/y_test.parquet")

print("Guardado en /artifacts")

Guardado en /artifacts
